In [2]:
#CONDA ENV pytorch1 (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
import process_functions as pf
import pyBigWig as bw
import torch
from enformer_pytorch import Enformer, GenomeIntervalDataset
import polars as pl

#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
print(f'Python version {sys.version}')
print(f'Python path {sys.executable}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python version 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
Python path /home/dburrows/mambaforge/envs/pytorch1/bin/python


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
class MultiInputDataset(Dataset):
    def __init__(self, tabular_data, images, labels):
        self.tabular_data = tabular_data
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.tabular_data)

    def __getitem__(self, idx):
        tabular_sample = torch.tensor(self.tabular_data.iloc[idx].values, dtype=torch.float)
        image = Image.open(self.images[idx]).convert('RGB')
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        
        # Preprocess image as needed, convert to tensor
        image_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = image_transform(image)

        return tabular_sample, image, label


In [ ]:
"""Dataset and DataLoader classes for Pytorch Lightning neural network models.

Dataset types:

- TabularDataset: for tabular data. Converts two pandas DataFrames (X and y)
	  to a Pytorch Dataset. Indices are used to match X and y.

- CoordScalingTabularDataset: for tabular data with coordinate scaling. All
	columns that contain 'coord' in the name will be scaled by a factor of
	100,000 (e.g. 600,000 becomes 6). This is useful for scaling the east
	and north coordinates in the UKBB dataset which range from about 0 to
	1.2 million.
"""



class episplice_dataset(Dataset):
    """
    Dataset object containing epi
    
    """

    def __init__(self,splice=None, mcg, mch, atac):
        """ Initialise epigenetic splice dataset """
        self.splice = splice
        self.mcg = mcg
        self.mch = mch
        self.atac = atac
        
    def __len__(self):
        """ Return length of dataset """
        return len(self.splice)
        
    def __getitem__(self,idx):
        """ Return splice, mcg, mch, atac at index """
        if self.y is not None:
            return(
            
            #process data -> split into features and take mean
                
            #return as a vector -> 
            
            )
        
#TEST!!

#DATALoader? 



class TabularDataset(Dataset):
	"""Tabular dataset for Pytorch neural network models.

	Args:
		X: pandas DataFrame, features
		y: pandas Series, target
	"""

	def __init__(self, X, y=None):
		"""Initialize tabular dataset."""
		self.X = X
		self.y = y

	def __len__(self):
		"""Return length of dataset."""
		return len(self.X)

	def __getitem__(self, idx):
		"""Return X and y at index."""
		if self.y is not None:
			return (
				torch.tensor(self.X.iloc[idx].values, dtype=torch.float32), 
				torch.tensor(self.y.iloc[idx].values, dtype=torch.float32)
			)
		else:
			return torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
		



In [3]:
"""Dataset and DataLoader classes for Pytorch Lightning neural network models.

Dataset types:

- TabularDataset: for tabular data. Converts two pandas DataFrames (X and y)
	  to a Pytorch Dataset. Indices are used to match X and y.

- CoordScalingTabularDataset: for tabular data with coordinate scaling. All
	columns that contain 'coord' in the name will be scaled by a factor of
	100,000 (e.g. 600,000 becomes 6). This is useful for scaling the east
	and north coordinates in the UKBB dataset which range from about 0 to
	1.2 million.
"""



class TabularDataset(Dataset):
	"""Tabular dataset for Pytorch neural network models.

	Args:
		X: pandas DataFrame, features
		y: pandas Series, target
	"""

	def __init__(self, X, y=None):
		"""Initialize tabular dataset."""
		self.X = X
		self.y = y

	def __len__(self):
		"""Return length of dataset."""
		return len(self.X)

	def __getitem__(self, idx):
		"""Return X and y at index."""
		if self.y is not None:
			return (
				torch.tensor(self.X.iloc[idx].values, dtype=torch.float32), 
				torch.tensor(self.y.iloc[idx].values, dtype=torch.float32)
			)
		else:
			return torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
		

class CoordScalingTabularDataset(TabularDataset):
	"""Tabular dataset for Pytorch neural network models with coordinate scaling.

	All columns that contain 'coord' in the name will be scaled by a factor of
	100,000 (e.g. 600,000 becomes 6). This is useful for scaling the east and
	north coordinates in the UKBB dataset which range from about 0 to 600,000.
	"""

	def __init__(self, X, y=None):
		"""Initialize tabular dataset."""
		# Scale coordinates
		for col in X.columns:
			if 'coord' in col:
				X[col] = X[col] / 100000

		super().__init__(X, y)

Note: you may need to restart the kernel to use updated packages.
